In [146]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import FixedLocator, FixedFormatter
from unidecode import unidecode
from tqdm import tqdm
import json
import re
from IPython.display import HTML
# import itertools
import warnings

<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
			font-weight: bolder;
	}
</style>
<h3>Funciones y variables globales</h3>

In [147]:
# archivo conteos y agrupaciones a doc. original
# wb_original_results = './Cambios_aplicados/wb_original_results.xlsx'
wb_final_results = './Cambios_aplicados/wb_final_results.xlsx'

# wb libro regs únicos 
wb_regs_unicos = './Cambios_aplicados/wb_regs_unicos.xlsx'

# libro a analizar
# variables files
file_to_analize = './Sitios SEMS.xlsx'

In [148]:
# funciones globales

def save_df_changes(df_object, file_name='Libro_cambios_aplicados'):
  """
  Save workbooks in the default path with xlsx extention
  
	Args:
			df_object (_type_): dataframe to save as file
			file_name (str, optional): filename NO extention
	"""
  try:
    df_object.to_excel(f'./Cambios_aplicados/{file_name}.xlsx', index=None)
    print(f'Se ha exportado el libro: {file_name}.xlsx\nRuta del archivo: "Cambios_aplicados -> {file_name}.xlsx"')
  except PermissionError:
    print("el libro no se pudo exportar, debido a que el libro exportado anteriormente esta abierto, se debe cerrar.")

def get_acronym(state_name):
  """
  get the acronym from the mexican state
  """
  tmp_res = None
  state_name = state_name.upper()
  try:
    stts_mexico = {
      'AGUASCALIENTES' : 'AGS.', 'BAJA CALIFORNIA' : 'B.C.', 'BAJA CALIFORNIA SUR' : 'B.C.S.', 'CAMPECHE' : 'CAMP.', 'COAHUILA DE ZARAGOZA' : 'COAH.', 'COLIMA' : 'COL.', 'CHIAPAS' : 'CHIS.', 'CHIHUAHUA' : 'CHIH.', 'CIUDAD DE MÉXICO' : 'C.D.M.X.', 'DURANGO' : 'DGO.', 'GUANAJUATO' : 'GTO.', 'GUERRERO' : 'GRO.', 'HIDALGO' : 'HGO.', 'JALISCO' : 'JAL.', 'MÉXICO' : 'MÉX.', 'MICHOACÁN DE OCAMPO' : 'MICH.', 'MORELOS' : 'MOR.', 'NAYARIT' : 'NAY.', 'NUEVO LEÓN' : 'N.L.', 'OAXACA' : 'OAX.', 'PUEBLA' : 'PUE.', 'QUERÉTARO' : 'QRO.', 'QUINTANA ROO' : 'Q. ROO.', 'SAN LUIS POTOSÍ' : 'S.L.P.', 'SINALOA' : 'SIN.', 'SONORA' : 'SON.', 'TABASCO' : 'TAB.', 'TAMAULIPAS' : 'TAMPS.', 'TLAXCALA' : 'TLAX.', 'VERACRUZ DE IGNACIO DE LA LLAVE' : 'VER.', 'YUCATÁN' : 'YUC.', 'ZACATECAS' : 'ZAC.'}
    tmp_res = stts_mexico[state_name]
    return tmp_res
  except KeyError:
    return 'SIN INFO.'

def get_dep_with_break(key_name):
  try:
    tmp_res = None
    key_name = key_name.upper()
    dic_deps = {
      'ESTABLECIMIENTO DE SALUD': 'ESTABL.\nSALUD',
      'ESTABLECIMIENTO DE EDUCACIÓN': 'ESTABL.\nEDUCACIÓN',
      'ESTABLECIMIENTO COMUNITARIO': 'ESTABL.\nCOMUNITARIO',
      'ESTABLECIMIENTO GOBIERNO': 'ESTABL.\nGOBIERNO'
    }
    tmp_res = dic_deps[key_name]
    return tmp_res
  except KeyError:
    return 'OTRA DEP.'

def sin_ascentos(nombre: str)-> str:
  """
  reemplazar tildes en palabras
  """
  abc = [
    ('Á', 'A'), ('É', 'E'), ('Í', 'I'), ('Ó', 'O'), ('Ú', 'U'), ('Ü', 'U'), 
    ('á', 'a'), ('é', 'e'), ('í', 'i'), ('ó', 'o'), ('ú', 'u'), ('ü', 'u')
    ]
  for a in abc:
    nombre = nombre.replace(a[0], a[1])
  return nombre

def save_df_in_sheet_from_workbook(dataframe, sheetname, workbookname):
  """
  Save a DataFrame into an Excel worksheet inside a specified Excel file.
  
  Arguments:
  - dataframe: The DataFrame to be saved into the Excel worksheet
  - sheetname: The name of the Excel worksheet in which the DataFrame will be written
  - workbookname: The name of the Excel file in which the DataFrame will be saved
  """
  try:
    with pd.ExcelWriter(workbookname, mode='a') as writer:
      dataframe.to_excel(writer, sheet_name=sheetname, index=None)
  except FileNotFoundError:
    with pd.ExcelWriter(workbookname) as writer:
      dataframe.to_excel(writer, sheet_name=sheetname, index=None)
  except ValueError:
    pass

# Función para validar latitudes
def validar_latitud(latitud):
  latitud = float(latitud)
  latitud_patron = r'^[-+]?([1-8]?\d(\.\d+)?|90(\.0+)?)$'
  return re.match(latitud_patron, str(latitud)) is not None

# Patrón de regex para validar longitudes decimales

# Función para validar longitudes
def validar_longitud(longitud):
  longitud = float(longitud)
  # longitud_patron = r'^[-+]?(180(\.0+)?|((1[0-7]\d)|([1-9]?\d))(\.\d+)?)$'
  longitud_patron = r'^-?\d+\.\d+$'
  return re.match(longitud_patron, str(longitud)) is not None

lst_reemplazos = [
  ('delegación ', ''), ('delegacion ', ''), ('ciudad ', ''), ('alcaldia ', ''), ('alcaldía ', ''), ('ciudad de ', ''), ('cd. ', ''), ('sta. ', 'santa '), ('gral. ', 'general'), ('dr. ', 'doctor'), ('ciudad del ', ''), ('ciudad de ', ''), ('san ', ''),( '(', ''),( ')', ''), ('francisco', ''), ('juárez', ''), ('juarez', ''), ('ejido ', ''), ('jardines', '')
  ]

In [149]:
# cargar en memoria el archivo de excel
data = pd.read_excel(io=file_to_analize, sheet_name=0, dtype={'Código postal*': object})

# crear dataframe
df = pd.DataFrame(data)

# eliminar data despues de convertirla en un dataframe
del data

In [150]:
# lr_df -> última fila de dataframe
lr = df.shape[0]
display(HTML(f'<h2 style="color: #fb8500;">El archivo tiene tiene {"{:,}".format(lr)} registros y {df.shape[1]} columnas</h2>'))

del lr

<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
			font-weight: bolder;
	}
</style>
<h3>Columnas Clave Dependencia (1) y Dependencia (2) </h3>

In [151]:
# importación del catálogo de ramos
tRamos = pd.read_csv('./Compact_inegi_dbLocs/ramo.csv')
lst_ramos = tRamos["cve_ramo"].to_list()
lst_desc_ramos = tRamos['descripcion'].to_list()
dictRamosByID = dict(zip(lst_ramos, lst_desc_ramos))
dictRamosByDesc = dict(zip(lst_desc_ramos, lst_ramos))

In [152]:
# actualizar la columna de clave dependencia y dependencia 

def update_row(row):
    
    clv = row['Clave de la Dependencia*']
    dep = row['Dependencia*']
    
    # si el registro esta en la lista de claves pero el valor de diccionario no corresponde a la dependencia
    if clv in lst_ramos and dictRamosByID[clv] != dep:
        row['Dependencia*'] = dictRamosByID[clv]
    
    # si la clave no esta en la lista de ramos y la clave no es un valor vacio
    elif clv not in lst_ramos and clv not in [' ', '']:
        try:
            row['Dependencia*'] = dictRamosByID[clv]
            row['Clave de la Dependencia*'] = dictRamosByDesc[dep]
        except KeyError:
            row['Clave de la Dependencia*'] = ' '
    return row

df = df.apply(update_row, axis=1)


<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
			font-weight: bolder;
	}
</style>
<h3>Columna Clave de inmueble (3) </h3>

In [153]:
# hacer el conteO de la clave inmueble, en primera instancia la clave de inmueble es una código único no vacio

# agrupacion de las claves y conteo de las repeticiones por cada clave
conteo_por_clvs = df.groupby(by='Clave del inmueble*').agg(regs_por_clave=('Clave del inmueble*', 'count')).reset_index()

# mantener solo las claves cuya repetición es mayor a una
conteo_por_clvs_duplicadas_clv =  conteo_por_clvs[conteo_por_clvs['regs_por_clave'] > 1].reset_index()

# segmentacion de conteo donde las claves repetidas estan vacias, solo saber el número de registros vacios
regs_sin_clv = conteo_por_clvs_duplicadas_clv[conteo_por_clvs_duplicadas_clv['Clave del inmueble*'] == " "]['regs_por_clave'][0]

# de las claves que no estan vacias mostrar las claves cuyo conteo es mayor a 1 y su valor es diferente a vacio [" " o ""]
regs_clv_dupl = conteo_por_clvs_duplicadas_clv[conteo_por_clvs_duplicadas_clv['Clave del inmueble*'] != " "]['regs_por_clave'].sum()

print(f'Registros SIN clave asignada: {regs_sin_clv}\nRegistros clave DUPLICADA: {regs_clv_dupl}\nTotal registros: {regs_sin_clv + regs_clv_dupl}')

Registros SIN clave asignada: 26
Registros clave DUPLICADA: 34
Total registros: 60


In [154]:
# agrupación de clave de inmueble y número de registros por clave
conteo_por_clvs_duplicadas_clv = conteo_por_clvs_duplicadas_clv[['Clave del inmueble*','regs_por_clave']]

# creación de una lista de todos los registros duplicados por clave y reemplazo de los vacios por un None
lst_dupli_clv = [x if x != " " and x != "" else None for x in conteo_por_clvs_duplicadas_clv['Clave del inmueble*'].to_list()]

lst_dupli_clv  = [elemento for elemento in lst_dupli_clv if elemento is not None]

# segmentación del dataframe, filtrar solo los registros que estan repetidos en su clave de inmueble en la lista de claves duplicadas
df_regs_duplicados_by_clv = df[df['Clave del inmueble*'].isin(lst_dupli_clv)].sort_values(by='Clave del inmueble*')

# eliminar variables
del tRamos, conteo_por_clvs_duplicadas_clv, lst_desc_ramos, conteo_por_clvs, df_regs_duplicados_by_clv

In [155]:
# validación y creación de estatus de duplicidad en doc. interno
df['status_intern_duplicidad'] = df['Clave del inmueble*'].apply(lambda x: 'clv interno duplicado' if x in lst_dupli_clv else 'clv interno no duplicado')

<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
		font-weight: bolder;
	}
</style>
<h3>Columna Tipo de inmuebles (5) </h3>

In [156]:
# importar catálogo de tipo de inmueble
tTipoInmueble = pd.read_csv('./Compact_inegi_dbLocs/tipo_inmueble.csv')

# crear lista de tipos de inmueble
lstTipoInmueble = tTipoInmueble['tipo_inmueble'].to_list()


In [157]:
def modificar_tipo_inmueble(clv_dep, dep, tp_inmueble):
  # si la clave de dependencias es igual a 11 y la dependencia es igual a EDUCACIÓN PÚBLICA 
  # entonces el tp_inmueble se reasigna a ESTABLECIMIENTO DE EDUCACIÓN
  if clv_dep == 11 and dep == "EDUCACIÓN PÚBLICA":
    tp_inmueble = "ESTABLECIMIENTO DE EDUCACIÓN"
  
  return tp_inmueble

# Aplica la función a todas las filas del dataframe
df["Tipo de inmuebles *"] = df.apply(lambda x: modificar_tipo_inmueble(x["Clave de la Dependencia*"], x["Dependencia*"], x["Tipo de inmuebles *"]), axis=1)


In [158]:
# eliminación de variables
del tTipoInmueble
del lstTipoInmueble
del dictRamosByDesc
del dictRamosByID
del regs_clv_dupl
del regs_sin_clv
del lst_dupli_clv
del lst_ramos

<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
		font-weight: bolder;
	}
</style>
<h3>Columna Subtipo de Inmueble (6) </h3>

In [159]:
#import catálogo de subtipos de inmueble
tSubtipoInmueble = pd.read_csv('./Compact_inegi_dbLocs/subtipo_inmueble.csv')

In [160]:

#               .d8888b.        d8888  .d8888b.   .d88888b.       888     888 888b    888 8888888 .d8888b.   .d88888b.  
#              d88P  Y88b      d88888 d88P  Y88b d88P" "Y88b      888     888 8888b   888   888  d88P  Y88b d88P" "Y88b 
#              888    888     d88P888 Y88b.      888     888      888     888 88888b  888   888  888    888 888     888 
#              888           d88P 888  "Y888b.   888     888      888     888 888Y88b 888   888  888        888     888 
#              888          d88P  888     "Y88b. 888     888      888     888 888 Y88b888   888  888        888     888 
#              888    888  d88P   888       "888 888     888      888     888 888  Y88888   888  888    888 888     888 
#              Y88b  d88P d8888888888 Y88b  d88P Y88b. .d88P      Y88b. .d88P 888   Y8888   888  Y88b  d88P Y88b. .d88P 
#               "Y8888P" d88P     888  "Y8888P"   "Y88888P"        "Y88888P"  888    Y888 8888888 "Y8888P"   "Y88888P"  

def upgrade_subtipo_inmueble(nombre_inmueble, subtipo_inmueble):
  # CASO ESPECIAL YA QUE EL NOMBRE DE LOS INMUEBLES COMO TAL ESTABAN SEGMENTADA 
  # UNA PARTE EN EL CAMPO NOMBRE DE INMUEBLE Y OTRA PARTE EN EL CAMPO SUBTIPO DE INMUEBLE
  
  # primera depuración para homologar nombres de inmuebles 
  
  if subtipo_inmueble == 'COLEGIO DE EDUCACIÓN PROFESIONAL TÉCNICA (CONALEP)':
    nombre_inmueble = f'{subtipo_inmueble} {nombre_inmueble}'

  elif subtipo_inmueble == 'CENTRO DE BACHILLERATO TECNOLÓGICO AGROPECUARIO (CBTA)':
    nombre_inmueble = f'CENTRO DE BACHILLERATO TECNOLÓGICO AGROPECUARIO - {nombre_inmueble}'

  elif subtipo_inmueble == 'EXTENSIÓN (CBTA)':
    nombre_inmueble = f'EXTENSIÓN {nombre_inmueble}'

  elif subtipo_inmueble == 'OFICINA ADMINISTRATIVA':
    nombre_inmueble = f'{nombre_inmueble} (Oficina administrativa)'

  elif subtipo_inmueble == 'CENTRO DE ESTUDIOS TECNOLÓGICOS DEL MAR (CETMAR)':
    nombre_inmueble = f'CENTRO DE ESTUDIOS TECNOLÓGICOS DEL MAR - {nombre_inmueble}'
  
  elif subtipo_inmueble == 'CENTRO DE ESTUDIOS DE BACHILLERATO (CEB)':
    nombre_inmueble = f'CENTRO DE ESTUDIOS DE BACHILLERATO - {nombre_inmueble}'
  
  elif subtipo_inmueble == 'CENTRO DE ESTUDIOS TECNOLÓGICOS EN AGUAS CONTINENTALES (CETAC)':
    nombre_inmueble = f'CENTRO DE ESTUDIOS TECNOLÓGICOS EN AGUAS CONTINENTALES - {nombre_inmueble}'
  
  elif subtipo_inmueble == 'EXTENSIÓN (CETMAR)':
    nombre_inmueble = f'EXTENSIÓN {nombre_inmueble}'
  
  elif subtipo_inmueble == 'CENTRO DE BACHILLERATO TECNOLÓGICO FORESTAL (CBTF)':
    nombre_inmueble = f'CENTRO DE BACHILLERATO TECNOLÓGICO FORESTAL - {nombre_inmueble}'
  
  elif subtipo_inmueble == 'COLEGIO DE EDUCACIÓN PROFESIONAL TÉCNICA DEL ESTADO DE TLAXCALA (CONALEP)':
    nombre_inmueble = f'COLEGIO DE EDUCACIÓN PROFESIONAL TÉCNICA DEL ESTADO DE TLAXCALA - {nombre_inmueble}'
  
  elif subtipo_inmueble == 'EXTENSIÓN CBTA' or subtipo_inmueble == 'EXTENSIÓN (CBTF)':
    nombre_inmueble = f'EXTENSIÓN - {nombre_inmueble}'
  
  elif subtipo_inmueble == 'UNIDADES DE CAPACITACIÓN PARA EL DESARROLLO RURAL (UNCADER)':
    nombre_inmueble = f'UNIDADES DE CAPACITACIÓN PARA EL DESARROLLO RURAL - {nombre_inmueble}'
  
  elif 'CENTRO DE ESTUDIOS TECNOLÓGICOS INDUSTRIAL Y DE SERVICIOS' in subtipo_inmueble:
    nombre_inmueble = f'CENTRO DE ESTUDIOS TECNOLÓGICOS INDUSTRIAL Y DE SERVICIOS -  {nombre_inmueble}'
  
  elif 'CENTRO DE BACHILLERATO TECNOLÓGICO INDUSTRIAL Y DE SERVICIOS' in subtipo_inmueble:
    nombre_inmueble = f'CENTRO DE BACHILLERATO TECNOLÓGICO INDUSTRIAL Y DE SERVICIOS - {nombre_inmueble}'

  elif subtipo_inmueble in ['CENTRO DE BACHILLERATO TECNOLÓGICO AGROPECUARIO No. 167', 'EXTENSIÓN (CETAC)', 'COLEGIO DE EDUCACIÓN PROFESIONAL TÉCNICA (CONALEP)', 'COLEGIO DE EDUCACIÓN PROFESIONAL TÉCNICA  DEL ESTADO DE TLAXCALA (CONALEP)', 'CENTRO MULTIMODAL DE ESTUDIOS CIENTIFICOS Y TECNOLOGICOS DEL MAR Y AGUAS CONTINENTALES', 'CENTRO DE INVESTIGACION DE RECURSOS NATURALES (CIRENA)', 'CENTRO DE BACHILLERATO TECNOLOGICO AGROPECUARIO NUM 191']:
    nombre_inmueble = f'{subtipo_inmueble} - {nombre_inmueble}'
  
  return nombre_inmueble

df['Nombre del inmueble*'] = df.apply(lambda x: upgrade_subtipo_inmueble(x['Nombre del inmueble*'], x['Subtipo de inmueble *']), axis=1)

In [161]:
lst_general_concepts_sub_tp_inmueble = []
lst_dicts_inmueble_wrong_good = {}

with open('./Compact_inegi_dbLocs/dict_subtp_inmueble.json', mode='r', encoding='utf-8') as f:
  contenido = f.read()
  f.close()
  dict_subtp_inmueble = json.loads(contenido)
  del contenido
  

for key in dict_subtp_inmueble.keys():
  for value in dict_subtp_inmueble[key]:
    lst_formas_str = [
      f'{value}'.lower(), 
      f'{value}'.upper(), 
      f'{value}'.title(), 
      value, 
      sin_ascentos(f'{value}'), 
      sin_ascentos(f'{value}'.lower()), 
      sin_ascentos(f'{value}'.upper()), 
      sin_ascentos(f'{value}'.title())
      ]
    
    for l in lst_formas_str:
      lst_dicts_inmueble_wrong_good[l] = key
      lst_general_concepts_sub_tp_inmueble.append(l)

del lst_formas_str

In [162]:
def get_right_subtp_inmueble(word):
  """
  Description: 
      search the word in a list of wrong words with wrong meaning to the field and get the correct value from the catalog
  
  Args:
      palabra (str): incorrect word to be search in the dictionary and get the right concept (subtipo inmueble)

  Returns:
      str: correct concept
  """
  if word in lst_general_concepts_sub_tp_inmueble:
    return lst_dicts_inmueble_wrong_good[word]
  else:
    return word
  
df['Subtipo de inmueble *'] = df['Subtipo de inmueble *'].apply(get_right_subtp_inmueble)

# eliminar variable 
del lst_general_concepts_sub_tp_inmueble
del lst_dicts_inmueble_wrong_good
del dict_subtp_inmueble

In [163]:
# conteo de registros por subtipos de inmuebles 
conteo_subtipo_inmueble = df.groupby(by='Subtipo de inmueble *').agg(conteo_regs=('Subtipo de inmueble *', 'count')).reset_index()
ordenado_conteo_subtipo_inmueble = conteo_subtipo_inmueble.sort_values(by='conteo_regs', ascending=False)

ordenado_conteo_subtipo_inmueble

,Subtipo de inmueble *,conteo_regs
1,PLANTEL EDUCACIÓN MEDIA SUPERIOR,1609
0,OFICINA ADMINISTRATIVA,57


In [164]:
# desechar variables que ya no se van a usar
del tSubtipoInmueble
del conteo_subtipo_inmueble
del ordenado_conteo_subtipo_inmueble


<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Columna Estado (8) </h3>

In [165]:
# importar catálogo de estado
cat_edos = pd.read_csv('./Compact_inegi_dbLocs/cat_edos.csv', encoding='utf-8')
cat_edos = cat_edos.rename(columns={'ï»¿id_edo': 'id_edo'})

In [166]:
def get_id_correct_edo_name(edo_name: str) -> int:
    """
    Description:
      get the id of the mexican state

    Args:
      edo_name (str): the name of the mexican state
      cat_edos (pd.DataFrame): a DataFrame with columns 'id_edo' and 'edo' containing the
                               mapping between state names and state ids

    Returns:
      int: the id of the mexican state, or None if the state name is not found
    """
    edo_name = edo_name.upper().strip()
    filtro = cat_edos[cat_edos['edo'] == edo_name]
    if filtro.shape[0] == 1:
        return filtro['id_edo'].iloc[0]
    else:
        temp_edo_name = None
        
        if edo_name in ['ESTADO DE MÉXICO', 'ESTADO DE MEXICO', 'MÉXICO']:
            temp_edo_name = "MÉXICO"
        
        elif edo_name in ['COAHUILA']:
            temp_edo_name = "COAHUILA DE ZARAGOZA" 
        
        elif edo_name in ['MICHOACAN', 'MICHOACÁN']:
            temp_edo_name = "MICHOACÁN DE OCAMPO"
        
        elif edo_name in ['VERACRUZ', 'VERACRUZ LLAVE']:
            temp_edo_name = "VERACRUZ DE IGNACIO DE LA LLAVE"
        
        elif edo_name in ['SAN LUIS POTOSI']:
            temp_edo_name = "SAN LUIS POTOSÍ"
        
        elif edo_name in ['QUERÉTARO DE ARTEAGA', 'QUERETARO DE ARTEAGA', 'QUERETARO', 'QUERÉTARO']:
            temp_edo_name = "QUERÉTARO" 
        
        # sección recursiva
        if temp_edo_name is not None:
            return get_id_correct_edo_name(temp_edo_name)
        else:
            return 0


In [167]:
# añadir la lista de id de cada uno de los estados
lst_ids = df['Estado*'].apply(get_id_correct_edo_name)
df.insert(7, 'id_edo', lst_ids)
del lst_ids

In [168]:
def get_edo_name_from_id_edo(id_edo: int) -> str:
  """
  Description:
    search the correct name in the mexican states directory
  Aguments:
    id_edo is a integer number from the mexican states directory 
  Return:
    the correct name in the mexican states directory
  """
  if id_edo != 0:
    filtro = cat_edos[cat_edos.id_edo == id_edo]
    if filtro.shape[0] == 1:
      return filtro['edo'].iloc[0]
    else:
      return 'INFO. SIN DEFINIR'
  else:
    return 'INFO. SIN DEFINIR'

# hacer correción en los nombres del campo estado
df['Estado*'] = df['id_edo'].apply(get_edo_name_from_id_edo)

In [169]:
# eliminación de algunas variables
del cat_edos

<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Columna Municipio (9) </h3>

In [170]:
#  importar base de datos municipios
cat_mun = pd.read_csv('./Compact_inegi_dbLocs/AGEEML_20233271140141.csv', encoding='utf8', usecols=['CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN'])
cat_mun = cat_mun.drop_duplicates()
cat_mun['NOM_ENT'] = cat_mun['NOM_ENT'].str.lower()
cat_mun['NOM_MUN'] = cat_mun['NOM_MUN'].str.lower()

In [171]:
# eliminar espacio
df['Municipio*'] = df['Municipio*'].str.strip()


In [172]:
# primera iteración tratando de encontrar los id de cada municipio en toda la base de datos de estado y municipio
def get_id_mun(id_edo: int, edo_name: str) -> int:
  """
  Description:
		get the id of the mun 
	Arguments:
    id_edo = integer id from mexican state
    edo_name = mexican state name
	Return:
    id mun
  """
  edo_name = edo_name.lower().strip()
  if id_edo != 0:
    filter =  cat_mun[(cat_mun.CVE_ENT == id_edo) & (cat_mun.NOM_MUN == edo_name)]
    if filter.shape[0] == 1:
      return filter['CVE_MUN'].iloc[0]
    else:
      return 0
  else:
    return 0

lst_id_mun = df.apply(lambda x: get_id_mun(x['id_edo'], x['Municipio*']), axis=1)
df.insert(9, 'id_mun', lst_id_mun)
del lst_id_mun

In [173]:
# hacer busqueda de municipios a partir de que en el filtrado aparezca una palabra similar a la que aparece en el documento que manda la dependencia
def get_id_mun_contains(id_edo: int, id_mun: int, mun_name: str) -> int:
  """
  Description:
    get the id of the mun by the name of the mexican state using the methond contains id the cat_mun field
  Arguments:
    id_edo = int, id of the mexican state
    id_mun = int, id of the mexican mun
    mun_name = str, mun_name 
  Return:
    id_mun with data type int
  """
  mun_name = mun_name.lower()
  if id_edo != 0 and id_mun == 0:
    filtro = cat_mun[(cat_mun['CVE_ENT'] == id_edo) & (cat_mun['NOM_MUN'].str.contains(mun_name))]
    if filtro.shape[0] == 1:
      return filtro['CVE_MUN'].iloc[0]
    else:
      return 0
  elif id_edo != 0 and id_mun != 0:
    return id_mun

df['id_mun'] = df.apply(lambda x: get_id_mun_contains(x['id_edo'], x['id_mun'], x['Municipio*']), axis=1)

In [174]:
# buscar por registros sin ascentos y unidecode
def unidecode_cat_mun(palabra):
  return unidecode(palabra)

# crear una columna con valores pasado por unidecode (borrar tildes y ñ)
lst_cat_mun_unidecode = cat_mun['NOM_MUN'].apply(unidecode)
cat_mun.insert(4, 'unidecode_mun', lst_cat_mun_unidecode)
del lst_cat_mun_unidecode

# crear columna con valores sin tildes
lst_cat_mun_sin_tilde = cat_mun['NOM_MUN'].apply(sin_ascentos)
cat_mun.insert(5, 'sin_ascentos_mun', lst_cat_mun_sin_tilde)
del lst_cat_mun_sin_tilde

In [175]:
# buscar el municipio en la cat_mun previo se crean dos columnas una con unidecode y otra sin ascentos para hacer coincidencias 
def get_id_mun_unidecode_sinTildes(id_edo: int, id_mun: int, num_name: str) -> int:
  """
  Descripcion: 
		Obtener el valor del municipio a partir de eliminar tildes en el nombre del municipio 
  Argumentos:
		id_edo = int, valor númerico de el id del estado
		id_mun = int, valor númerico del id del municipio
		mun_name = str, cadena de texto del nombre del municipio
  Retorno:
		el id del municipio siempre y cuando de origen sea 0 y tambien el filtrado de información retorne al menos un valor
  """
  
  num_name = num_name.lower()
  for x in lst_reemplazos:
    num_name = num_name.replace(x[0], x[1])

  num_name = num_name.strip()
  
  filtro = cat_mun[
    (cat_mun['CVE_ENT'] == id_edo) & 
    (
      (cat_mun['NOM_MUN'].str.contains(num_name)) | 
      (cat_mun['unidecode_mun'].str.contains(unidecode(num_name))) |
      (cat_mun['sin_ascentos_mun'].str.contains(sin_ascentos(num_name)))
      ) 
    ]
  if filtro.shape[0] == 1 and id_edo > 0 and id_mun == 0:
    return filtro['CVE_MUN'].iloc[0]
  else:
    return id_mun

# aplicación de la función get_id_mun_unidecode_sinTildes
df['id_mun'] = df.apply(lambda x: get_id_mun_unidecode_sinTildes(x['id_edo'], x['id_mun'], x['Municipio*']), axis=1)

In [176]:
# crear matriz cadenas  de texto e iterarlas para encontrar una coincidencia que devuelva solo un registro
def get_id_mun_iter_str_mun(id_edo: int, id_mun: int, mun_name: str) -> int:
  """
  Descripcion:
    obtener el id de el municipio a partir del nombre, para ello se itera como lista el nombre del municipio y si el filtro da un resultado se retornara el id de ese sitio de lo contrario debera dejarlo en cero
  Argumentos:
    id_edo: int, es el id del estado
    id_mun: int, es el id del municipio 
    mun_name: str, cadena de texto del nombre del municipio a ser convertida como cadena de texto
  Return:
    retorna el id del nombre del municpio 
  """
  if id_edo > 0 and id_mun == 0:
    mun_name = mun_name.lower()
    mun_name = mun_name.split(' ')
    mun_name = list(sorted(mun_name, key=len, reverse=True))
    len_mun_name = len(mun_name)

    for x in enumerate(mun_name):
      filtro = cat_mun[
        (cat_mun['CVE_ENT'] == id_edo) & 
        (
          (cat_mun['NOM_MUN'].str.contains(x[1])) | 
          (cat_mun['unidecode_mun'].str.contains(unidecode(x[1]))) | 
          (cat_mun['sin_ascentos_mun'].str.contains(sin_ascentos(x[1]))) 
        )
        ]
      
      if x[0] +1 < len_mun_name:
        if filtro.shape[0] == 1:
          return filtro['CVE_MUN'].iloc[0]
          break
      
      elif x[0]+1 == len_mun_name:
        if filtro.shape[0] == 1:
          return filtro['CVE_MUN'].iloc[0]
          break
        else:
          return 0
  else:
    return id_mun

# aplicar la función iterar valores de cadena de texto para obtener coincidencias
df['id_mun'] = df.apply(lambda x: get_id_mun_iter_str_mun(x['id_edo'], x['id_mun'], x['Municipio*']), axis=1)

In [177]:
# coincidencia exacta
# crear matriz cadenas  de texto e iterarlas para encontrar una coincidencia que devuelva solo un registro
def get_id_mun_iter_str_mun_coincidencia_exacta(id_edo: int, id_mun: int, mun_name: str) -> int:
  """
  Descripcion:
    obtener el id de el municipio a partir del nombre, para ello se itera como lista el nombre del municipio y si el filtro da un resultado se retornara el id de ese sitio de lo contrario debera dejarlo en cero
  Argumentos:
    id_edo: int, es el id del estado
    id_mun: int, es el id del municipio 
    mun_name: str, cadena de texto del nombre del municipio a ser convertida como cadena de texto
  Return:
    retorna el id del nombre del municpio 
  """
  if id_edo > 0 and id_mun == 0:
    mun_name = mun_name.lower()
    mun_name = mun_name.split(' ')
    mun_name = list(sorted(mun_name, key=len, reverse=True))
    len_mun_name = len(mun_name)

    for x in enumerate(mun_name):
      filtro = cat_mun[
        (cat_mun['CVE_ENT'] == id_edo) & 
        (
          (cat_mun['NOM_MUN'] == x[1]) | 
          (cat_mun['unidecode_mun'] == unidecode(x[1])) | 
          (cat_mun['sin_ascentos_mun'] == sin_ascentos(x[1])) 
        )
        
        ]
      
      if x[0] +1 < len_mun_name:
        if filtro.shape[0] == 1:
          return filtro['CVE_MUN'].iloc[0]
          break
      
      elif x[0]+1 == len_mun_name:
        if filtro.shape[0] == 1:
          return filtro['CVE_MUN'].iloc[0]
          break
        else:
          return 0
  else:
    return id_mun

# aplicar la función iterar valores de cadena de texto para obtener COINCIDENCIAS EXACTAS
df['id_mun'] = df.apply(lambda x: get_id_mun_iter_str_mun_coincidencia_exacta(x['id_edo'], x['id_mun'], x['Municipio*']), axis=1)


In [178]:
# hacer la correción en el nombre de la localidad con base en el catálogo
def get_mun_correct_name(id_edo: int, id_mun: int, mun_name: str) -> str:
  if id_edo != 0 and id_mun != 0:
    filtro = cat_mun[(cat_mun['CVE_ENT'] == id_edo) & (cat_mun['CVE_MUN'] == id_mun)]
    if filtro.shape[0] == 1:
      return f"{filtro['NOM_MUN'].iloc[0]}".title()
    else:
      return mun_name
  else:
    return mun_name

df['Municipio*'] = df.apply(lambda x: get_mun_correct_name(x['id_edo'], x['id_mun'], x['Municipio*']), axis=1)

# eliminar de memoria el catalogo de municipios
del cat_mun

<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Columna Localidades (10) </h3>

In [179]:
# Limpiar la columna 'id_mun' si es necesario
df['id_mun'] = df['id_mun'].fillna(0) # Rellenar valores nulos con 0 o cualquier otro valor predeterminado
df['id_mun'] = pd.to_numeric(df['id_mun'], errors='coerce') # Convertir a tipo numérico

# Convertir la columna 'id_mun' a valores enteros y asignar el resultado a una nueva variable o a la misma columna
df['id_mun'] = df['id_mun'].astype(int) # Convertir a valores enteros

df['Localidad*'] = df['Localidad*'].str.lower()

In [180]:
# quitar espacios inecesarios y reemplazar algunas palabras
with warnings.catch_warnings():
  warnings.filterwarnings('ignore')
  for x in lst_reemplazos:
    df['Localidad*'].str.replace(x[0], x[1])
    
df['Localidad*'] = df['Localidad*'].str.strip()

In [181]:

def replace_val_locs(nom_localidad):
  lst_reemplazos_2 = [
    ('alcaldía ', ''), ('delegación ', '')
  ]
  for x in lst_reemplazos_2:
    nom_localidad = nom_localidad.replace(x[0], x[1])
    nom_localidad = nom_localidad.replace(sin_ascentos(x[0]), sin_ascentos(x[1]))
  return nom_localidad

df['Localidad*'] = df['Localidad*'].apply(replace_val_locs)

In [182]:
# importar el catálogo de localidades
cat_locs = pd.read_csv('./Compact_inegi_dbLocs/AGEEML_20233271140141.csv', usecols=['NOM_ENT','CVE_ENT', 'CVE_MUN', 'NOM_MUN', 'CVE_LOC', 'NOM_LOC'])
cat_locs = cat_locs.drop_duplicates()
cat_locs['NOM_LOC'] = cat_locs['NOM_LOC'].str.lower()
cat_locs['unidecode_locs'] = cat_locs['NOM_LOC'].apply(unidecode)
cat_locs['sinTilde_locs'] = cat_locs['NOM_LOC'].apply(sin_ascentos)

# quitar espacios inecesarios
cat_locs['unidecode_locs'] = cat_locs['unidecode_locs'].str.strip()
cat_locs['sinTilde_locs'] = cat_locs['sinTilde_locs'].str.strip()

In [183]:
# crear primera iteración para encontrar la coincidencia exacta a partir del id edo, id mun y el nombre de la localidad, 
# SE GENERA LA COLUMNA DEL ID DE LA LOCALIDAD
def get_id_loc(id_edo : int, id_mun : int, loc_name: str) -> int:
  """
  Descripcion: 
    con esta funcion se obtiene el id de la localidad con ayuda de el id del estado, id del municipio y el nombre de la localidad
  Argumentos:
    id_loc: int, número entero que corresponde al id de la localidad
    id_mun: int, número entero que corresponde al id del municipio
    loc_name: str, cadena de texto que corresponde al nombre de la localidad
  Retorno:
    retorna el id de la localidad
  """
  loc_name = loc_name.lower()
  
  loc_name = loc_name.strip()
  
  filtro = cat_locs[
    (cat_locs['CVE_ENT'] == id_edo) & 
    (cat_locs['CVE_MUN'] == id_mun) & 
    (
      (cat_locs['NOM_LOC'] == loc_name) |
      (cat_locs['unidecode_locs'] == unidecode(loc_name)) | 
      (cat_locs['sinTilde_locs'] == sin_ascentos(loc_name))
      )
    ]
  
  if filtro.shape[0] == 1:
    return filtro['CVE_LOC'].iloc[0]
  elif loc_name == ' ' or loc_name == '':
    return 0
  else:
    return 0

# aplicación de la función get_id_loc
lst_id_locs = df.apply(lambda x: get_id_loc(x['id_edo'], x['id_mun'], x['Localidad*']), axis=1)
df.insert(11, 'id_loc', lst_id_locs)

# eliminar la variable de la memoria
del lst_id_locs

In [184]:
# encontrar un match a partir del id del estado y del id de la localidad siempre y cuando se tenga el id del estado y el id de la localidad sea igual a cero
def get_correction_mun_loc_ids(id_edo: int, id_mun: int, id_loc: int, loc_name: str) -> tuple:
  """
  Descripcion: 
		Obtener el id de la localidad a partir del id del estado y el nombre de la localidad, si el filtrado da un resltado entonces se debe de hacer la correción
  """
  if id_edo != 0 and id_loc == 0 and len(loc_name) != 0:
    filtro = cat_locs[
			(cat_locs.CVE_ENT == id_edo) &
			(
			(cat_locs.NOM_LOC == loc_name) |
			(cat_locs.unidecode_locs == unidecode(loc_name)) |
			(cat_locs.sinTilde_locs == sin_ascentos(loc_name))
			)
		]
    
    if filtro.shape[0] == 1:
      corr_id_mun = filtro['CVE_MUN'].iloc[0]
      corr_id_loc = filtro['CVE_LOC'].iloc[0]
      return corr_id_mun, corr_id_loc
    else:
      return (id_mun, id_loc)
  else:
    return (id_mun, id_loc)

# aplicación de la función get_correction_mun_loc_ids
lst_mun_locs_ids = df.apply(lambda x: get_correction_mun_loc_ids(x['id_edo'], x['id_mun'], x['id_loc'], x['Localidad*']), axis=1)

# segmentación de la información por columna, deconstrucción
df['id_mun'], df['id_loc'] = lst_mun_locs_ids.apply(lambda x: x[0]), lst_mun_locs_ids.apply(lambda x: x[1])

# eliminación de la variable
del lst_mun_locs_ids

In [185]:
def get_correction_mun_loc_ids_contains(id_edo: int, id_mun: int, id_loc: int, loc_name: str) -> tuple:
  """
  Descripcion: 
		obtener correción del id del municipio y localidad si el filtrado de id edo y nombre de la localidad retorna un valor que contenga el nombre de la localidad de la tabla dentro del catálogo
  """
  if id_edo != 0 and id_loc == 0:
    with warnings.catch_warnings():
      warnings.filterwarnings('ignore')
      filtro = cat_locs[
        (cat_locs.CVE_ENT == id_edo) & 
        (
          (cat_locs.NOM_LOC.str.contains(loc_name)) | 
          (cat_locs.unidecode_locs.str.contains(unidecode(loc_name))) |
          (cat_locs.sinTilde_locs.str.contains(sin_ascentos(loc_name)))
      )
      ]
      if filtro.shape[0] == 1:
        corr_id_mun = filtro['CVE_MUN'].iloc[0]
        corr_id_loc = filtro['CVE_LOC'].iloc[0]
        return (corr_id_mun, corr_id_loc)
      else:
        return (id_mun, id_loc)
  else:
    return (id_mun, id_loc)
    
# aplicación de la función
corr_id_mun_loc_contains = df.apply(lambda x: get_correction_mun_loc_ids_contains(x['id_edo'], x['id_mun'], x['id_loc'], x['Localidad*']), axis=1)

# segmentar info por columnas:
df['id_mun'], df['id_loc'] = corr_id_mun_loc_contains.apply(lambda x: x[0]), corr_id_mun_loc_contains.apply(lambda x: x[1])

# eliminación de la variable 
del corr_id_mun_loc_contains

In [186]:
def get_corr_id_mun_loc_substr(id_edo: int, id_mun: int, id_loc: int, loc_name: str) -> tuple:
  """
  Descripcion:
    obtener el id del estado en caso de que el nombre de la localidad tenga el caracter '(' en su nombre, en caso de que tenga ese caracter se aplica un substring se filtra por estado y el resultado de la extracción de la izquierda y en caso de que el filtrado regrese un registro se toma el id del municipio y la localidad del filtro
  """
  if id_edo != 0 and id_loc == 0 and len(loc_name) > 0:

    if '(' in loc_name:
      parentesis = loc_name.find('(')
      loc_name = loc_name[:parentesis].strip()
      
      filtro = cat_locs[
      (cat_locs.CVE_ENT == id_edo) & 
      (
        (cat_locs.NOM_LOC == loc_name) |
        (cat_locs.unidecode_locs == unidecode(loc_name)) |
        (cat_locs.sinTilde_locs == sin_ascentos(loc_name))
      )
      ]
      
      if filtro.shape[0] == 1:
        corr_id_mun = filtro['CVE_MUN'].iloc[0]
        corr_id_loc = filtro['CVE_LOC'].iloc[0]
        return (corr_id_mun, corr_id_loc)
      else:
        return (id_mun, id_loc)
    else:
      return (id_mun, id_loc)
  else:
    return (id_mun, id_loc)

# aplicación de la función
lst_corr_id_mun_loc_substr = df.apply(lambda x: get_corr_id_mun_loc_substr(x['id_edo'], x['id_mun'], x['id_loc'], x['Localidad*']), axis=1)

# reemplazo de los valores en columnas id_mun y id_loc
df['id_mun'], df['id_loc'] = lst_corr_id_mun_loc_substr.apply(lambda x: x[0]), lst_corr_id_mun_loc_substr.apply(lambda x: x[1])

# elinminacion de la variable
del lst_corr_id_mun_loc_substr

In [187]:
# esta función solo le debe de aplicar a los registros cuyo nombre de localidad aparece mas de una vez al momento de 
# hacer el filtrado por el id del estado, el id del municipio y el nombre de la localidad.
# OBTENER LA LOCALIDAD CON REGISTRO DE ID MÁS CHICO DE LOS RESULTADOS FILTRADOS

def get_smaller_id_loc_names_repited(id_edo: int, id_mun: int, id_loc: int, loc_name: str) -> int:
  """
  Descripcion:
		obtener el id más pequeño del filtrado siempre y cuando haya mas de una coincidencia exacta al momento de filtrar la base de datos de los catálogos utilizando el id del estado, id del municipio y el nombre de la localidad
  Argumentos:
		id_edo: int, es el número de índice en la bd de cat. localidades
		id_mun: int, es el número de indice del municipio en la bd de localidades
		id_loc: int, es el número de indice de la localidad en la bd de localidades
		loc_name: str, es el nombre de la localidad
  Resultado:
		retorna el id de la localidad siempre y cuando haya mas de in registro cuyo nombre de la localidad es repetido en id edo, id municipio y nombre de la localidad
  """
  if id_edo != 0 and id_mun != 0 and id_loc == 0 and loc_name not in [" ", ""]:
    
    loc_name = loc_name.lower()
    
    for x in lst_reemplazos:
      loc_name = loc_name.replace(x[0], x[1])
    
    loc_name = loc_name.strip()
    
    filtro = cat_locs[
      (
        (cat_locs['NOM_LOC'] == loc_name) |
				(cat_locs['unidecode_locs'] == unidecode(loc_name)) |
				(cat_locs['sinTilde_locs'] == sin_ascentos(loc_name)) ) & 
      (cat_locs['CVE_ENT'] == id_edo) &
			(cat_locs['CVE_MUN'] == id_mun)]
    
    if filtro.shape[0] > 1:
      filtro.sort_values(by='CVE_LOC')
      return filtro['CVE_LOC'].iloc[0]
    else:
      return id_loc
  else:
    return id_loc

# aplicar la función de busqueda
df['id_loc'] = df.apply(lambda x: get_smaller_id_loc_names_repited(x['id_edo'], x['id_mun'], x['id_loc'], x['Localidad*']), axis=1)

In [188]:
# opción de include e iterar resultados:
def get_smaller_id_loc_names_contains(id_edo: int, id_mun: int, id_loc: int, loc_name: str) -> int:
    """
    Descripcion:
        obtener el id más pequeño del filtrado siempre y cuando haya mas de una coincidencia exacta al momento de filtrar la base de datos de los catálogos utilizando el id del estado, id del municipio y el nombre de la localidad
    Argumentos:
        id_edo: int, es el número de índice en la bd de cat. localidades
        id_mun: int, es el número de indice del municipio en la bd de localidades
        id_loc: int, es el número de indice de la localidad en la bd de localidades
        loc_name: str, es el nombre de la localidad
    Resultado:
        retorna el id de la localidad siempre y cuando haya mas de in registro cuyo nombre de la localidad es repetido en id edo, id municipio y nombre de la localidad
    """
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        
        if id_edo != 0 and id_mun != 0 and id_loc == 0 and loc_name not in [" ", ""]:
            loc_name = loc_name.lower()
            
            for x in lst_reemplazos:
                loc_name = loc_name.replace(x[0], x[1])
            
            loc_name = loc_name.strip()
            
            filtro = cat_locs[
                (
                    (cat_locs['NOM_LOC'].str.contains(loc_name)) |
                    (cat_locs['unidecode_locs'].str.contains(loc_name)) |
                    (cat_locs['sinTilde_locs'].str.contains(sin_ascentos(loc_name)))
                    ) &
                (cat_locs['CVE_ENT'] == id_edo) &
                (cat_locs['CVE_MUN'] == id_mun)]
            if filtro.shape[0] > 1:
                filtro.sort_values(by='CVE_LOC')
                return filtro['CVE_LOC'].iloc[0]
            else:
                return id_loc
        else:
            return id_loc

# aplicar la función de busqueda
df['id_loc'] = df.apply(lambda x: get_smaller_id_loc_names_contains(x['id_edo'], x['id_mun'], x['id_loc'], x['Localidad*']), axis=1)

In [189]:
# 16	MICHOACÁN DE OCAMPO	108	Zamora	0	ARIO DE RAYÓN
def get_id_loc_iter_vals_contains(id_edo: int, id_mun: int, id_loc: int, loc_name: str) -> int:
  """
  Descripcion:
    obtener el id del nombre de la localidad a partir de iterar el resultado de aplicar el filtro de aplicar por id estado, id municipio.
  Argumentos:
    id_edo: int, indice del estado
    id_mun: int, indice del municipio a partir del estado
    id_loc: int, indice de la localidad
    nom:loc: str, nombre de la localidad
  Retorno:
  """
  if id_edo != 0 and id_mun != 0 and id_loc == 0 and id_edo != 9 and loc_name not in [" ", ""]:
    loc_name = loc_name.lower()
    
    for x in lst_reemplazos:
      loc_name = loc_name.replace(x[0], x[1])
      
    loc_name = loc_name.strip()
    
    lst_loc_name = list(sorted(loc_name.split(" "), key=len, reverse=True))
    len_lst_loc_name = len(lst_loc_name)

    for x in enumerate(lst_loc_name):
      iter = x[0]
      palabra = x[1]
      filtro = cat_locs[
        (cat_locs.CVE_ENT == id_edo) & 
        (cat_locs.CVE_MUN == id_mun) & 
        (
          (cat_locs.NOM_LOC.str.contains(palabra)) | 
          (cat_locs.unidecode_locs.str.contains(palabra)) | 
          (cat_locs.sinTilde_locs.str.contains(palabra))
          )
        ]

      if iter + 1 < len_lst_loc_name:
        if filtro.shape[0] == 1:
          return filtro['CVE_LOC'].iloc[0]
          break
      else:
        if filtro.shape[0] == 1:
          return filtro['CVE_LOC'].iloc[0]
          break
        else:
          return id_loc
  else:
    return id_loc

# aplicación de la función
df['id_loc'] = df.apply(lambda x: get_id_loc_iter_vals_contains(x['id_edo'], x['id_mun'], x['id_loc'], x['Localidad*']), axis=1)

# eliminación de la lista de reemplazos
del lst_reemplazos

<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Estado, Municipio y Localidad</h3>

In [190]:
def get_name_mun_loc(id_edo: int, edo: str, id_mun: int, mun:str, id_loc: int, loc: str) -> tuple:
  """
  Descripción:
		obtener el nombre del municipio y la localidad siempre y cuando ninguno de los id sea igual a cero
	Argumentos:
		id_edo: int, id del estado con base en el catálogo de localidades
		id_mun: int, id del municipio con base en el catálogo de localidades
		id_loc: int, id del localidad con base en el catálogo de localidades
  """
  if id_edo != 0 and id_mun != 0 and id_loc != 0:
    filtro = cat_locs[
      (cat_locs.CVE_ENT == id_edo) &
      (cat_locs.CVE_MUN == id_mun) & 
      (cat_locs.CVE_LOC == id_loc)]
    if filtro.shape[0] == 1:
      edo = f"{filtro['NOM_ENT'].iloc[0]}".title()
      mun = f"{filtro['NOM_MUN'].iloc[0]}".title()
      loc = f"{filtro['NOM_LOC'].iloc[0]}".title()
      return (edo, mun, loc)
    else:
      return (f'{edo}'.title(), f'{mun}'.title(), f'{loc}'.title())
  else:
    return (f'{edo}'.title(), f'{mun}'.title(), f'{loc}'.title())
  

# generar lista de tuplas con correcciones en el nombre del estado, municipio y la localidad en los casos que aplica
lst_reeplace_names_edo_mun_loc = df.apply( lambda x: get_name_mun_loc( x['id_edo'], x['Estado*'], x['id_mun'], x['Municipio*'], x['id_loc'], x['Localidad*']), axis=1  )

# vaciado de la información a sus respectivas columnas
df['Estado*'] = lst_reeplace_names_edo_mun_loc.apply(lambda x: x[0])
df['Municipio*'] = lst_reeplace_names_edo_mun_loc.apply(lambda x: x[1])
df['Localidad*'] = lst_reeplace_names_edo_mun_loc.apply(lambda x: x[2])

# eliminación de variables
del lst_reeplace_names_edo_mun_loc
del cat_locs

<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Columna Nombre de asentamiento humano y C.P. </h3>

In [191]:
# importar el catálogo de códigos postales de sepomex
cat_cp = pd.read_csv('./Compact_inegi_dbLocs/cp_sepomex.csv', dtype={'d_codigo': object}, usecols=['d_codigo', 'd_asenta', 'd_tipo_asenta'])

In [192]:
# eliminar espacios inescerarios de la columna de colonia
df['Nombre de asentamiento humano (colonia)*'] = df['Nombre de asentamiento humano (colonia)*'].str.strip()

In [193]:
def get_cp(col_name: str, col_cp: str) -> str:
  """
  Descripción:
    obtener nombre del asentamiento humano siempre y cuando el que viene definido cae es ciertos supuestos y el filtrado retorna al menos un resultado
  """
  if col_name  in ['Nombre de asentamiento humano (colonia)', '', ' ', 'Nombre de asentamiento humano', 'asentamiento humano'] and len(f'{col_cp}') == 5:
    
    filtro = cat_cp[cat_cp.d_codigo == f'{col_cp}']
    
    if filtro.shape[0] != 0:
      return filtro['d_asenta'].iloc[0]
    else:
      return col_name
  else:
    return col_name

df['Nombre de asentamiento humano (colonia)*'] = df.apply(lambda x: get_cp(x['Nombre de asentamiento humano (colonia)*'], x['Código postal*']), axis=1)

# eliminación de la variable cat_cp
del cat_cp

<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Columna Latitud y Longitud</h3>

In [198]:
df['Latitud*'].astype(float)

ValueError: could not convert string to float: ''

In [196]:

# si_lat = 0
# no_lat = 0
def status_latitud(latitud):
  # global si_lat
  # global no_lat
  if validar_latitud(latitud):
    # si_lat += 1
    return latitud
  else:
    # no_lat += 1
    return f'[No Lat] {str(latitud)}'

# aplicación de la validación de las latitudeds
df['Latitud*'] = df['Latitud*'].apply(status_latitud)


ValueError: could not convert string to float: ''

In [ ]:
# si_lon = 0
# no_lon = 0
def status_longitud(longitud):
  # global si_lon
  # global no_lon
  if validar_latitud(longitud):
    # si_lon += 1
    return longitud
  else:
    # no_lon += 1
    return f'[No Lon] {longitud}'

# aplicación de la función para validar longitud
df['Longitud*'] = df['Longitud*'].apply(status_longitud)

<style>
	h3{
		background-color: red;
		color: white;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Localidades Prioritarias PROGRAMA DE COBERTURA SOCIAL</h3>

In [ ]:
cat_locs_pcs = pd.read_csv('./Compact_inegi_dbLocs/locs_prioritarias_programa_cobertura_social.csv', 
                           encoding='utf-8', 
                           usecols=['id_edo', 'id_mun', 'id_loc'], 
                           dtype={'id_edo': object, 'id_mun': object, 'id_loc': object})

cat_locs_pcs['code_edo_mun_loc'] = cat_locs_pcs['id_edo'] + " | " + cat_locs_pcs['id_mun'] + " | " + cat_locs_pcs['id_loc']

cat_locs_pcs = cat_locs_pcs.drop(columns=['id_edo', 'id_mun', 'id_loc'])

list_locs_prio_pcs = cat_locs_pcs['code_edo_mun_loc'].to_list()

del cat_locs_pcs

def is_loc_priority(id_edo: int, id_mun: int, id_loc: int) -> str:
  code = '{} | {} | {}'.format(id_edo, id_mun, id_loc)
  if code in list_locs_prio_pcs:
    return 'Prioritaria'
  return 'No Prioritaria'


In [ ]:
df['status_prioridad'] = df.apply(lambda x: is_loc_priority(x['id_edo'], x['id_mun'], x['id_loc']), axis=1)
del list_locs_prio_pcs

<style>
	h3{
		background-color: red;
		color: white;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Población femenina y masculina por localidad CENSO 2020</h3>

In [ ]:
censo = pd.read_csv('./Compact_inegi_dbLocs/conjunto_de_datos_iter_00CSV20.csv', low_memory=False, usecols=['ENTIDAD', 'MUN', 'LOC', 'POBTOT', 'POBFEM', 'POBMAS'])

censo = censo[
	(censo.ENTIDAD != 0) &
	(censo.MUN != 0) &
	(censo.LOC != 0)
]

# reemplazo de valores con * por un cero 0
censo = censo.replace({'POBFEM': '*', 'POBMAS': '*'}, 0)


In [ ]:
def get_population_by_loc(id_edo: int, id_mun: int, id_loc: int) -> tuple:
  """
  Descripción:
		obtener el número de la población femenina y masculina a partir d<style>
	h3{
		background-color: red;
		color: white;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Población femenina por localidad CENSO 2020</h3>el id del estado, id munucipio y el id de la localidad
		la tupla que retorna es = (población femenina, población masculina, porcentaje pop femenina, porcentaje pop masculina)
  """
  if id_edo != 0 and id_mun != 0 and id_loc != 0:
    filtro = censo[
			(censo.ENTIDAD == id_edo) &
			(censo.MUN == id_mun) &
			(censo.LOC == id_loc)
		]
    if filtro.shape[0] == 1:
      pob_fem = int(filtro['POBFEM'].iloc[0])
      pob_mas = int(filtro['POBMAS'].iloc[0])
      try: 
        pop_tot = pob_fem + pob_mas
        por_fem = round((pob_fem / pop_tot) * 100, 2)
        por_mas = round((pob_mas / pop_tot) * 100, 2)
        return (pob_fem, pob_mas, por_fem, por_mas )
      except (ValueError, ZeroDivisionError):
        return (pob_fem, pob_mas, 0, 0)
    else:
      return (0, 0, 0, 0)
  else:
    return (0, 0, 0, 0)

# aplicación de la función 
lst_tuplas_pop_por_loc = df.apply(lambda x: get_population_by_loc(x['id_edo'], x['id_mun'], x['id_loc']), axis=1)

# creación de las nuevas columnas
df['pob_femenina'] =  lst_tuplas_pop_por_loc.apply(lambda x: x[0])
df['pob_masculina'] =  lst_tuplas_pop_por_loc.apply(lambda x: x[1])
df['porcentaje_pop_fem']  =  lst_tuplas_pop_por_loc.apply(lambda x: x[2])
df['porcentaje_pop_mas']  =  lst_tuplas_pop_por_loc.apply(lambda x: x[3])

# eliminar la variable
del lst_tuplas_pop_por_loc
del censo

<style>
	h3{
		background-color: red;
		color: white;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Base de datos CFE</h3>

In [ ]:
# cargar la base de datos de CFE
data_cfe = pd.read_excel('./Compact_inegi_dbLocs/avance_ordenes_cfe_teit.xlsx', usecols=['entidad_id', 'municipio_id', 'localidad_clave', 'inmueble_clave', 'inmueble_nombre', 'conexion_estatus'], dtype={'inmueble_clave': object, 'inmueble_nombre': object, 'conexion_estatus': object})

# eliminar los espacios inecesarios de la base de datos de cfe en su columna clave de inmueble
data_cfe['inmueble_clave'] = data_cfe['inmueble_clave'].str.strip()

# convertir el nombre del sitio a minuscula
data_cfe['inmueble_nombre'] = data_cfe['inmueble_nombre'].str.lower()

# solo obtener los registros los cuales tienen un estatus de "instalada"
df_instaladas = data_cfe[data_cfe['conexion_estatus'] == 'Instalada']

# eliminar la base de datos de cfe para ahoorar espacio en memoria y solo usar el df de instalados
del data_cfe

# crear una lista de claves de sitios instalados, primero quitamos duplicados y eliminamos vacios 
lst_instaladas = df_instaladas['inmueble_clave'].drop_duplicates().dropna()

# reasignamos el valor de lst_instaladas para solo mantener los registros cuyo largo de información sea mayor a 5:
lst_instaladas = [f"{x}".upper() for x in lst_instaladas if len(x) > 5]


In [ ]:
def is_cfe_connected(clv_inmueble: str) -> str:
  if clv_inmueble in lst_instaladas:
    return 'Conectada'
  return 'No conectada'

# aplicación de la función para conocer el status de la conexión
df['cfe_conexion_status'] = df['Clave del inmueble*'].apply(is_cfe_connected)

# eliminar variable de lista de instaladas claves
del lst_instaladas

In [ ]:
# lst_temp_clvs = []
# def generate_key_installed(id_edo, id_mun, id_loc, name_site):
#   global lst_temp_clvs
#   temp = "{} | {} | {} | {}".format(id_edo, id_mun, id_loc, name_site)
#   lst_keys = [f'{temp}'.lower(), f'{temp}'.upper(), f'{temp}'.title(), sin_ascentos(f'{temp}'), unidecode(f'{temp}')]
#   for x in lst_keys:
#     lst_temp_clvs.append(x)

# df_instaladas.apply(lambda x: generate_key_installed(x['entidad_id'], x['municipio_id'], x['localidad_clave'], x['inmueble_nombre']), axis=1)

# LO DE ARRIBA NO FUNCIONA ECHARLE UN OJO Y COMPRENDERLO

def generate_key_installed(id_edo, id_mun, id_loc, name_site):
  temp = "{} | {} | {} | {}".format(id_edo, id_mun, id_loc, name_site)
  # lst_keys = [f'{temp}'.lower(), f'{temp}'.upper(), f'{temp}'.title()]
  lst_keys = [f'{temp}'.lower(), f'{temp}'.upper(), f'{temp}'.title(), sin_ascentos(f'{temp}'.lower()), unidecode(f'{temp}'.lower())]
  # Assuming sin_ascentos is a function defined elsewhere
  lst_keys += [sin_ascentos(f'{temp}'), unidecode(f'{temp}')]
  return lst_keys

lst_temp_clvs = []
for _, row in df_instaladas.iterrows():
  lst_temp_clvs += generate_key_installed(row['entidad_id'], row['municipio_id'], row['localidad_clave'], row['inmueble_nombre'])
  
del df_instaladas


In [ ]:
def validate_key_in_lst_clvs(id_edo, id_mun, id_loc, site_name, status_conection):
  if status_conection == 'No conectada':
    site_name = site_name.lower()
    temp_sin_ascentos  = "{} | {} | {} | {}".format(id_edo, id_mun, id_loc, sin_ascentos(site_name))
    temp_unidecode  = "{} | {} | {} | {}".format(id_edo, id_mun, id_loc, unidecode(site_name))
    temp_lower = "{} | {} | {} | {}".format(id_edo, id_mun, id_loc, site_name.lower())
    if (temp_sin_ascentos in lst_temp_clvs) or (temp_unidecode in lst_temp_clvs) or (temp_lower in lst_temp_clvs):
      return 'Conectada'
    return status_conection
  return status_conection

# aplicación de la función
df['cfe_conexion_status'] = df.apply(lambda x: validate_key_in_lst_clvs(x['id_edo'], x['id_mun'], x['id_loc'], x['Nombre del inmueble*'], x['cfe_conexion_status']), axis=1)

# eliminar la lista de claves
del lst_temp_clvs

<style>
	h3{
		background-color: red;
		color: white;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Base de Datos Preliminar PCSP</h3>

In [ ]:
db_pcsp = pd.read_csv('./Compact_inegi_dbLocs/respaldo_copia_rusp.csv', low_memory=False, encoding='utf-8', usecols=['Clave del inmueble*', 'Nombre del inmueble*', 'Id_Edo', 'ID_Mun', 'ID_Loc'])

In [ ]:
# eliminar espacios en columnas de nombre del inmueble y clave del inmueble
db_pcsp['Clave del inmueble*'] = db_pcsp['Clave del inmueble*'].str.strip()
db_pcsp['Nombre del inmueble*'] = db_pcsp['Nombre del inmueble*'].str.strip()


In [ ]:
lst_clvs_pcsp = db_pcsp[db_pcsp['Clave del inmueble*'].str.len() > 4]['Clave del inmueble*'].drop_duplicates().to_list()

In [ ]:
def validar_clv_in_preliminar_pcsp(clave):
  if clave in lst_clvs_pcsp:
    return 'SI'
  return 'NO'

# aplicación de la función validación en la base de datos preliminar pcsp
df['en_preliminar_pcsp'] = df['Clave del inmueble*'].apply(validar_clv_in_preliminar_pcsp)

# eliminación de la variable lista lst_clvs_pcsp
del lst_clvs_pcsp
del db_pcsp

<style>
	h3{
		background-color: orange;
		color: black;
		text-align: center;
		border: 2px solid red;
	}
</style>
<h3>Exportaciones sobre archivo original</h3>

In [ ]:
# 					╔═╗═╗ ╦╔═╗╔═╗╦═╗╔╦╗╔═╗╦═╗  ╔═╗╦═╗╔═╗╦ ╦╦╦  ╦╔═╗  ╔═╗╦═╗╦╔═╗╦╔╗╔╔═╗╦    ╔═╗╔═╗╔╗╔  ╔═╗╦═╗╦ ╦╔═╗╔═╗╔═╗
# 					║╣ ╔╩╦╝╠═╝║ ║╠╦╝ ║ ╠═╣╠╦╝  ╠═╣╠╦╝║  ╠═╣║╚╗╔╝║ ║  ║ ║╠╦╝║║ ╦║║║║╠═╣║    ║  ║ ║║║║  ║  ╠╦╝║ ║║  ║╣ ╚═╗
# 					╚═╝╩ ╚═╩  ╚═╝╩╚═ ╩ ╩ ╩╩╚═  ╩ ╩╩╚═╚═╝╩ ╩╩ ╚╝ ╚═╝  ╚═╝╩╚═╩╚═╝╩╝╚╝╩ ╩╩═╝  ╚═╝╚═╝╝╚╝  ╚═╝╩╚═╚═╝╚═╝╚═╝╚═╝
save_df_changes(df, 'archivo_original_cruces_aplicados')

In [ ]:
#           ╔═╗═╗ ╦╔═╗╔═╗╦═╗╔╦╗╔═╗╦═╗  ╔═╗╦═╗╔═╗╦ ╦╦╦  ╦╔═╗  ╔═╗╦╔╗╔  ╔╦╗╦ ╦╔═╗╦  ╦╔═╗╔═╗╔╦╗╔═╗╔═╗
#           ║╣ ╔╩╦╝╠═╝║ ║╠╦╝ ║ ╠═╣╠╦╝  ╠═╣╠╦╝║  ╠═╣║╚╗╔╝║ ║  ╚═╗║║║║   ║║║ ║╠═╝║  ║║  ╠═╣ ║║║ ║╚═╗
#           ╚═╝╩ ╚═╩  ╚═╝╩╚═ ╩ ╩ ╩╩╚═  ╩ ╩╩╚═╚═╝╩ ╩╩ ╚╝ ╚═╝  ╚═╝╩╝╚╝  ═╩╝╚═╝╩  ╩═╝╩╚═╝╩ ╩═╩╝╚═╝╚═╝

#           ╔╗╔╔═╗  ╔═╗╔═╗╔╗╔╔═╗╔═╗╔╦╗╔═╗╔╦╗╔═╗  ╔═╗╔═╗╔═╗  ╔╦╗╔═╗╦╔╦╗
#           ║║║║ ║  ║  ║ ║║║║║╣ ║   ║ ╠═╣ ║║║ ║  ║  ╠╣ ║╣    ║ ║╣ ║ ║ 
#           ╝╚╝╚═╝  ╚═╝╚═╝╝╚╝╚═╝╚═╝ ╩ ╩ ╩═╩╝╚═╝  ╚═╝╚  ╚═╝   ╩ ╚═╝╩ ╩ 

#           ╔╗╔╔═╗  ╔═╗╔╗╔  ╔═╗╦═╗╔═╗╦  ╦╔╦╗╦╔╗╔╔═╗╦═╗  ╔═╗╔═╗╔═╗╔═╗
#           ║║║║ ║  ║╣ ║║║  ╠═╝╠╦╝║╣ ║  ║║║║║║║║╠═╣╠╦╝  ╠═╝║  ╚═╗╠═╝
#           ╝╚╝╚═╝  ╚═╝╝╚╝  ╩  ╩╚═╚═╝╩═╝╩╩ ╩╩╝╚╝╩ ╩╩╚═  ╩  ╚═╝╚═╝╩  

# segmentación de registros no conectados con base en el doc de cfe teit y que no esten en la base de datos preliminar de pcsp
archivo_segmentado  = df[(df['cfe_conexion_status'] == 'No conectada') & (df['en_preliminar_pcsp'] == 'NO')]

# primer intento para eliminar registros duplicados que sean exactamente igual en cada uno de los campos
archivo_segmentado = archivo_segmentado.drop_duplicates()

# filtrar solo registros duplicados
duplicados_inter = archivo_segmentado[(archivo_segmentado['status_intern_duplicidad'] == 'clv interno duplicado')]

# iterar la lista de filas dependiendo del número de las mismas
lr_duplicados_inter = duplicados_inter.shape[0]

# lista donde se van almacenar los números de caracteres por registro para poder eliminar los más bajos en números de registros a partir de la clave repetida
lens_duplicados_inter = []

# iterar todas las filas y todas las columnas del documento de filtro repetidos internos para obtener el conteo de caracteres por cada repetido
# iterar filas
for r in range(0, lr_duplicados_inter):
  # número de caractéres
  w = 0
  # iterar columnas
  for c in range(0, 29):
    # incremento de conteo de caracteres
    w += len(f'{duplicados_inter.iloc[r,c]}')
  # agregar conteo a lista 
  lens_duplicados_inter.append(w)

# añadir el conteo de caracteres al subfiltro
duplicados_inter.insert(2, 'len_row_record', lens_duplicados_inter)

# segmentar subfiltro por el conteo de caracteres y la claves duplicadas en subfiltro
duplicados_inter = duplicados_inter[['len_row_record', 'Clave del inmueble*']]

# crear ordenado por clave (de forma alfabética) y el conteo de caracteres (descendente)
# duplicados_inter.sort_values(by=['len_row_record', 'Clave del inmueble*'], ascending=[False, False])

# crear lista de indices del dataframe contenedor
list_indices = []

# bucle para obtener el índice de cada registro que esta duplicado en sus claves
for r in range(0, lr_duplicados_inter):
  
  len_record = duplicados_inter['len_row_record'].iloc[r]
  clv = duplicados_inter['Clave del inmueble*'].iloc[r]
  
  # filtrado para obtener 
  indice = duplicados_inter[
    ( duplicados_inter['len_row_record'] == len_record) &
    ( duplicados_inter['Clave del inmueble*'] == clv )
    ].index.tolist()
  
  list_indices.append(indice[0])

# insertar la lista de indices en el subfiltrado
duplicados_inter.insert(0, 'indice_container', list_indices)

# crear lista de solo duplicados 
lst_duplicados = list(set(duplicados_inter['Clave del inmueble*'].to_list()))

# lst_index_to_del
lst_index_to_del = []

# bucle para iterar el subfiltro y solo contemplar para su eliminación cada registro que este por debajo del registro que tenga más contenido
for ele in lst_duplicados:
  subfiltro = duplicados_inter[duplicados_inter['Clave del inmueble*'] == ele].sort_values(by='len_row_record', ascending=False)
  for r in subfiltro['indice_container'].iloc[1:]:
    lst_index_to_del.append(r)

# reasignar la segmentación donde solo se queden los archivos que no estan repetidos, que no estan en la base de datos preliminar pcsp y que no aparezcan en la lista de conectados de CFE TEIT
archivo_segmentado = archivo_segmentado.drop(index= lst_index_to_del)

# guardar el archivo segmentado con registros unicos
save_df_changes(archivo_segmentado, 'archivo_unicos_NO_pcsp_NO_cfe')

# eliminar variables de la memoria
del duplicados_inter, lr_duplicados_inter, lens_duplicados_inter, list_indices, lst_duplicados, lst_index_to_del, subfiltro


In [ ]:
# eliminar columas que no se ocupan para las gráficas PARA AHORRAR ESPACIO EN MEMORIA
col_eliminables = ['Clave de la Dependencia*', 'Descripción de la ubicación', 'Tipo de vialidad*', 'Nombre de vialidad*', 'Número exterior*', 'Número interior*', 'Nombre de asentamiento humano (colonia)*', 'Código postal*', 'Luz eléctrica*', 'Material del Techo del inmueble', 'Nombre completo del responsable del sitio*', 'Correo institucional*', 'número teléfónico', 'Prioridad establecida', 'status_intern_duplicidad', 'status_prioridad', 'pob_femenina', 'pob_masculina', 'porcentaje_pop_fem', 'porcentaje_pop_mas', 'cfe_conexion_status', 'en_preliminar_pcsp']

df.drop(columns=col_eliminables)

archivo_segmentado.drop(columns=col_eliminables)


In [ ]:
#                ╔═╗╔═╗╔╗╔╔╦╗╔═╗╔═╗  ╦═╗╔═╗╔═╗╦╔═╗╔╦╗╦═╗╔═╗╔═╗  ╔═╗╔═╗╦═╗  ╔═╗╔═╗╔╦╗╔═╗╔╦╗╔═╗
#                ║  ║ ║║║║ ║ ║╣ ║ ║  ╠╦╝║╣ ║ ╦║╚═╗ ║ ╠╦╝║ ║╚═╗  ╠═╝║ ║╠╦╝  ║╣ ╚═╗ ║ ╠═╣ ║║║ ║
#                ╚═╝╚═╝╝╚╝ ╩ ╚═╝╚═╝  ╩╚═╚═╝╚═╝╩╚═╝ ╩ ╩╚═╚═╝╚═╝  ╩  ╚═╝╩╚═  ╚═╝╚═╝ ╩ ╩ ╩═╩╝╚═╝
# Calvin S
# https://patorjk.com/software/taag/#p=display&f=Calvin%20S&t=CONTEO%20REGISTROS%20POR%20ESTADO

display(HTML('<h2 style="background-color: #ff7d00; color: white;" >Los sitios que en el campo de "Estado*" no hayan tenido información en automático se clasifican con el concepto de "SIN INFORMACIÓN", los registros eliminados caen en las siguientes condiciones: <br><ol><li>Son registros que según el archivo de CFE TEIT ya tienen estatus de conectado</li><li>El sitio ya estaba previamente contemplado en preliminar del PCSP</li><li>El registro estaba duplicado dentro del mismo archivo que manda la dependencia o estado</li></ol></h3>'))

# conteo de sitios por estado de la república ARCHIVO ORIGINAL
conteo_regs_por_edo_original = df.groupby(by='Estado*').agg(conteo_x_edo_original=('Estado*', 'count')).reset_index().sort_values(by='conteo_x_edo_original', ascending = False)

# conteo de sitios por estado de la república ARCHIVO SEGMENTADO
conteo_regs_por_edo_segmentado = archivo_segmentado.groupby(by='Estado*').agg(conteo_x_edo_segmentado=('Estado*', 'count')).reset_index().sort_values(by='conteo_x_edo_segmentado', ascending=False)

# unión de los dataframe conteo por estado
merge_conteo_x_edo = pd.merge(left=conteo_regs_por_edo_original, right=conteo_regs_por_edo_segmentado, on='Estado*')

# añadir acrónimo de estado para ahorrar espacio en el gráfico
merge_conteo_x_edo.insert(0, 'acronimo', merge_conteo_x_edo['Estado*'].apply(get_acronym))

# eliminar la columna de estado para ahorrar espacio
merge_conteo_x_edo = merge_conteo_x_edo.drop(columns=['Estado*'])

# ordenar de forma descendente a partir de la columna del conteo por estado de referencia dek archivo original
merge_conteo_x_edo = merge_conteo_x_edo.sort_values(by='conteo_x_edo_original', ascending=False)

count_x_edo_original, count_x_edo_segmentado = '{:,}'.format(merge_conteo_x_edo['conteo_x_edo_original'].sum()), '{:,}'.format(merge_conteo_x_edo['conteo_x_edo_segmentado'].sum())

# guardar archivo con el dataframe de conteo de registros por estado
save_df_in_sheet_from_workbook(merge_conteo_x_edo, 'cont_regs_x_edo', wb_final_results)

# hacer una nueva columna con el estatus de la variación
merge_conteo_x_edo['hay_cambios'] = merge_conteo_x_edo['conteo_x_edo_original'] != merge_conteo_x_edo['conteo_x_edo_segmentado']

# solo seleccionar los registros que tienen alguna variación en sus registros al momento de ser depurados
merge_conteo_x_edo = merge_conteo_x_edo[merge_conteo_x_edo['hay_cambios'] == True]

# eliminar la columna bool cambios
merge_conteo_x_edo = merge_conteo_x_edo.drop(columns=['hay_cambios'])

# --------------- CREACIÓN DE GRÁFICA -------------------------------

# crear instancia de gráfica
chrt_merge_conteo_x_edo = merge_conteo_x_edo.plot(
  kind='bar', figsize=(20, 5), title= f'conteo sitios por estado', width = 0.8, xlabel= 'Estado de la república mexicana', ylabel= 'Sitios por estado', color = ['#691C32', '#BC955C']
  )

# cambiar el tamaño de la fuente del las etiquetas de las barras del eje x
chrt_merge_conteo_x_edo.set_xticklabels(labels=merge_conteo_x_edo['acronimo'], fontsize=9)

for x in chrt_merge_conteo_x_edo.containers:
  chrt_merge_conteo_x_edo.bar_label(x, fontsize=9)

grupo_nombres = [f'Sitios en archivo original({count_x_edo_original})', f'Sitios en archivo segmentado ({count_x_edo_segmentado})']
chrt_merge_conteo_x_edo.legend(grupo_nombres, loc='best')

plt.show()

# elimar variables:
del conteo_regs_por_edo_original, conteo_regs_por_edo_segmentado, merge_conteo_x_edo, chrt_merge_conteo_x_edo, grupo_nombres

# # --------------- CREACIÓN DE GRÁFICA -------------------------------

# # Crear una figura y un eje
# fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(20, 12))

# # Establecer el título del gráfico
# [ax[lbl[0]].set_title(lbl[1], fontsize=10) for lbl in enumerate([f'Conteo de sitios por estado ORIGINAL ({count_x_edo_original} sitios)', f'Conteo de sitios por estado SEGMENTADO ({count_x_edo_segmentado} sitios)'])]

# # graficar subplots (2)
# [merge_conteo_x_edo.plot(kind='bar', ax=ax[lbl[0]], y=lbl[1]) for lbl in enumerate(['conteo_x_edo_original', 'conteo_x_edo_segmentado'])]

# # Agregar etiquetas al subplot
# [ax[x].set_xticklabels(merge_conteo_x_edo['acronimo'], fontsize=10) for x in range(0,2)]

# # # Eliminar las etiquetas del eje x en ambos subplots
# # # ax[0].set_xticklabels([])
# # ax[1].set_xticklabels([])

# # Añadir los valores a las columnas subplot 0
# for i in range(0,2):
#     for x in ax[i].containers:
#         ax[i].bar_label(x)

# # mostrar la figura
# plt.show()


In [ ]:
#           ╔═╗╔═╗╔╗╔╔╦╗╔═╗╔═╗  ╔═╗╦╔╦╗╦╔═╗╔═╗  ╔═╗╔═╗╦═╗  ╔╦╗╔═╗╔═╗╔═╗╔╗╔╔╦╗╔═╗╔╗╔╔═╗╦╔═╗
#           ║  ║ ║║║║ ║ ║╣ ║ ║  ╚═╗║ ║ ║║ ║╚═╗  ╠═╝║ ║╠╦╝   ║║║╣ ╠═╝║╣ ║║║ ║║║╣ ║║║║  ║╠═╣
#           ╚═╝╚═╝╝╚╝ ╩ ╚═╝╚═╝  ╚═╝╩ ╩ ╩╚═╝╚═╝  ╩  ╚═╝╩╚═  ═╩╝╚═╝╩  ╚═╝╝╚╝═╩╝╚═╝╝╚╝╚═╝╩╩ ╩

# agrupación de registros por estado y tipo de dependencia [ARCHIVO ORIGINAL Y SEGMENTADO]
conteo_dep_archivo_original = df['Tipo de inmuebles *'].value_counts().rename_axis('dependencia').reset_index(name='conteo_original')
conteo_dep_archivo_segmentado = archivo_segmentado['Tipo de inmuebles *'].value_counts().rename_axis('dependencia').reset_index(name='conteo_segmentado')

# merge conteo de archivo original y segmentado
merge_conteo_regs_x_dep = pd.merge(left=conteo_dep_archivo_original, right=conteo_dep_archivo_segmentado, on='dependencia')

# guardar la segmentación de los datos
save_df_in_sheet_from_workbook(merge_conteo_regs_x_dep, 'sitios_x_dep', wb_final_results)

# añadir el nombre corto por dependencia para ahorrar espacio en gráfica
merge_conteo_regs_x_dep.insert(0,'shrt_name_dep',  merge_conteo_regs_x_dep['dependencia'].apply(get_dep_with_break)) 

# eliminar la columna de dependencia para ahorrar espacio en df
merge_conteo_regs_x_dep = merge_conteo_regs_x_dep.drop(columns=['dependencia'])

# sumatoria
ttl_regs_dep_original, ttl_regs_dep_segmentado = merge_conteo_regs_x_dep['conteo_original'].sum(), merge_conteo_regs_x_dep['conteo_segmentado'].sum() 

# ***************************** GRÁFICA *********************************************************************

chrt_merge_conteo_regs_x_dep = merge_conteo_regs_x_dep.plot(
  kind='bar',   figsize=(20, 5),  title= f'conteo sitios por tipo dependencia',  width = 0.8,
  xlabel= 'Dependencia',  ylabel= 'Sitios por dependencia',  color = ['#691C32', '#BC955C'])

# cambiar el tamaño de la fuente del las etiquetas de las barras del eje x
chrt_merge_conteo_regs_x_dep.set_xticklabels(labels=merge_conteo_regs_x_dep['shrt_name_dep'], fontsize=9)

# añadir valores a las columnas 
for x in chrt_merge_conteo_regs_x_dep.containers:
  chrt_merge_conteo_regs_x_dep.bar_label(x, fontsize=9)

grupo_nombres = [f'Sitios en archivo ORIGINAL ({ttl_regs_dep_original})', f'Sitios en archivo SEGMENTADO ({ttl_regs_dep_segmentado})']
chrt_merge_conteo_regs_x_dep.legend(grupo_nombres, loc='best')

plt.show()

del chrt_merge_conteo_regs_x_dep, conteo_dep_archivo_original, conteo_dep_archivo_segmentado, ttl_regs_dep_original, ttl_regs_dep_segmentado, grupo_nombres, merge_conteo_regs_x_dep


In [ ]:
#				 ╔═╗╦╔╗╔  ╔═╗╦  ╔═╗╦  ╦╔═╗╔═╗  ╔═╗  ╔╦╗╦ ╦╔═╗╦  ╦╔═╗╔═╗╔╦╗╔═╗╔═╗  ╔═╗╔╗╔  ╔╦╗╦╔═╗╔╦╗╔═╗  ╔═╗╦═╗╔═╗╦ ╦╦╦  ╦╔═╗
#				 ╚═╗║║║║  ║  ║  ╠═╣╚╗╔╝║╣ ╚═╗  ║ ║   ║║║ ║╠═╝║  ║║  ╠═╣ ║║╠═╣╚═╗  ║╣ ║║║  ║║║║╚═╗║║║║ ║  ╠═╣╠╦╝║  ╠═╣║╚╗╔╝║ ║
#				 ╚═╝╩╝╚╝  ╚═╝╩═╝╩ ╩ ╚╝ ╚═╝╚═╝  ╚═╝  ═╩╝╚═╝╩  ╩═╝╩╚═╝╩ ╩═╩╝╩ ╩╚═╝  ╚═╝╝╚╝  ╩ ╩╩╚═╝╩ ╩╚═╝  ╩ ╩╩╚═╚═╝╩ ╩╩ ╚╝ ╚═╝

# df['Clave del inmueble*'].value_counts().rename_axis('Clave').reset_index(name='Conteo')
# df['Clave del inmueble*'].value_counts().reset_index().rename(columns={'index': 'Clave', 'Clave del inmueble*': 'Conteo'})
# df['Clave del inmueble*'].value_counts().rename('Conteo').reset_index(name='Clave')
# df['Clave del inmueble*'].value_counts().to_frame('Conteo').reset_index().rename(columns={'index': 'Clave'})
# conteo_claves = df['Clave del inmueble*'].value_counts().rename('Conteo').reset_index()

display(HTML('<h2 style="background-color: #ff7d00; color: white;" >Las claves que se duplican puede variar, una clave puede repetirse al menos 2 veces.</h3>'))

# conteo de claves
conteo_claves = df['Clave del inmueble*'].value_counts().rename_axis('Clave').reset_index(name='Conteo')

# eliminación de espacios
conteo_claves['Clave'] = conteo_claves['Clave'].str.strip()

sin_clv = conteo_claves[conteo_claves['Clave'] == '']['Conteo'].sum()
clv_unicas = conteo_claves[(conteo_claves['Conteo'] == 1) & (conteo_claves['Clave'] != '')]['Conteo'].sum()
clv_duplicadas = conteo_claves[(conteo_claves['Conteo'] > 1) & (conteo_claves['Clave'] != '')]


# guardar los registros que tienen las claves repetidas 
save_df_in_sheet_from_workbook(dataframe= clv_duplicadas, sheetname='clvs_duplicadas', workbookname=wb_final_results)

# claves duplicadas se reasigna valor 
clv_duplicadas = clv_duplicadas['Conteo'].sum()

# data para crear un nuevo dataframe
data_conte_status_clvs = {
  'status': ['sin_clave', 'duplicadas', 'unicas'],
  'conteo': [sin_clv, clv_duplicadas, clv_unicas] }

total_regs = '{:,}'.format(sum([sin_clv, clv_duplicadas, clv_unicas]))

# dataframe conteo por status de claves
df_conteo_clv = pd.DataFrame(data_conte_status_clvs)

# convertir el valor "conteo" a numérico
df_conteo_clv['conteo'] = df_conteo_clv['conteo'].astype(int)

# ordenado del df status claves por conteo de forma descendente
df_conteo_clv = df_conteo_clv.sort_values(by='conteo', ascending=False)

# ****************************************************************************************

# Crear una figura y un eje
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))

# Establecer el título del gráfico
ax[0].set_title(f'Conteo por status de claves ({total_regs})', fontsize=10)
ax[1].set_title(f'Porcentaje por status de claves ({total_regs})', fontsize=10)

# graficar el primer subplot como un gráfico de barras
df_conteo_clv.plot(kind='bar', ax=ax[0])

# Agregar etiquetas al eje x del primer subplot
ax[0].set_xticklabels(df_conteo_clv['status'], fontsize=10)

# graficar el segundo subplot como un gráfico de pastel
df_conteo_clv.plot(kind='pie', y='conteo', labels=df_conteo_clv['status'], ax=ax[1], autopct='%1.1f%%', textprops={'fontsize': 10})

# Eliminar las etiquetas del eje x en ambos subplots
# ax[0].set_xticklabels([])
ax[1].set_xticklabels([])

# Añadir los valores a las columnas subplot 0
for x in ax[0].containers:
  ax[0].bar_label(x)

# mostrar la figura
plt.show()

# eliminar variables
del conteo_claves, sin_clv, clv_unicas, clv_duplicadas, data_conte_status_clvs, total_regs, df_conteo_clv, fig, ax

In [ ]:
#		 ╦  ╔═╗╔═╗╔═╗╦  ╦╔╦╗╔═╗╔╦╗╔═╗╔═╗  ╔═╗╦╔╗╔  ╦╔╦╗    ╦╔╗╔╔═╗╔═╗╦
#		 ║  ║ ║║  ╠═╣║  ║ ║║╠═╣ ║║║╣ ╚═╗  ╚═╗║║║║  ║ ║║    ║║║║║╣ ║ ╦║
#		 ╩═╝╚═╝╚═╝╩ ╩╩═╝╩═╩╝╩ ╩═╩╝╚═╝╚═╝  ╚═╝╩╝╚╝  ╩═╩╝────╩╝╚╝╚═╝╚═╝╩

display(HTML('<h2 style="background-color: #ff7d00; color: white; text-align: center;">La razón por la que no se puede asignar el identificador clave (ID) es por que el nombre de la localidad que registran en el archivo no corresponde con los que vienen en la base de datos del INEh</h2>'))

# creación del conteo por filtro
locs_sin_id_x_edo_original = df[(df['id_loc'] == 0)]['Estado*'].value_counts().rename_axis('Estado').reset_index(name='conteo_original')
locs_sin_id_x_edo_segmentado = archivo_segmentado[(archivo_segmentado['id_loc'] == 0)]['Estado*'].value_counts().rename_axis('Estado').reset_index(name='conteo_segmentado')


# unión de los dataframes agrupados por tipo de archivo
merge_locs_sin_id_x_edo = pd.merge(left=locs_sin_id_x_edo_original, right=locs_sin_id_x_edo_segmentado, on='Estado')

# insertar columna de acrónimo para ahorrrar espacio en gráfico
merge_locs_sin_id_x_edo.insert(0, 'acronimo', merge_locs_sin_id_x_edo['Estado'].apply(get_acronym) )

# eliminar la columna de estado para ahorrar espacio en df
merge_locs_sin_id_x_edo = merge_locs_sin_id_x_edo.drop(columns=['Estado'])

# guardar el archivo unido
save_df_in_sheet_from_workbook(merge_locs_sin_id_x_edo, 'locs_sin_id_loc', wb_final_results)

# totales por archivo, localidades sin id
ttl_merge_locs_sin_id_x_edo_original = '{:,}'.format(merge_locs_sin_id_x_edo['conteo_original'].sum())
ttl_merge_locs_sin_id_x_edo_segmentado = '{:,}'.format(merge_locs_sin_id_x_edo['conteo_segmentado'].sum())
# ------------------------ CREACIÓN DE GRÁFICO -----------------------------------------------

chrt_merge_locs_sin_id_x_edo = merge_locs_sin_id_x_edo.plot(
  kind='bar',   figsize=(20, 5),  title= 'Sitios no localizados por estado',  width = 0.8,
  xlabel= 'Edo. de la República',  ylabel= 'Sitios por estado',  color = ['#691C32', '#BC955C'])

# cambiar el tamaño de la fuente del las etiquetas de las barras del eje x
chrt_merge_locs_sin_id_x_edo.set_xticklabels(labels=merge_locs_sin_id_x_edo['acronimo'], fontsize=10)

# añadir valores a las columnas 
for x in chrt_merge_locs_sin_id_x_edo.containers:
  chrt_merge_locs_sin_id_x_edo.bar_label(x, fontsize=9)

# personalizar los leyendas 
grupo_nombres = [f'Sitios en locs. sin ID en archivo ORIGINAL ({ttl_merge_locs_sin_id_x_edo_original})', f'Sitios en locs. sin ID en archivo SEGMENTADO ({ttl_merge_locs_sin_id_x_edo_segmentado})']
chrt_merge_locs_sin_id_x_edo.legend(grupo_nombres, loc='best')

plt.show()

# eliminación de variables
del locs_sin_id_x_edo_original, locs_sin_id_x_edo_segmentado, merge_locs_sin_id_x_edo, ttl_merge_locs_sin_id_x_edo_original, ttl_merge_locs_sin_id_x_edo_segmentado, chrt_merge_locs_sin_id_x_edo, grupo_nombres

In [ ]:
display(HTML('<h2 style="background-color: #ff7d00; color: white; text-align: center;">Considerar que hay sitios en localidades que no se les pudo asignar id de la localidad por lo tanto no se pudieron casificar si son prioritarias o no prioritarias</h2>'))

# crear dataframe conteo de sitios no prioritarios
sitios_locs_no_prio_original = df[df['status_prioridad'] == 'Prioritaria']['Estado*'].value_counts().rename_axis('estado').reset_index(name='conteo_prioritarias_original')
sitios_locs_no_prio_segmentado = archivo_segmentado[(archivo_segmentado['status_prioridad'] == 'Prioritaria')]['Estado*'].value_counts().rename_axis('estado').reset_index(name='conteo_prioritarias_segmentado')

# unión de los df de archivo original y de segmentación
merge_sitios_locs_prio = pd.merge(left=sitios_locs_no_prio_original, right=sitios_locs_no_prio_segmentado, on='estado')

# agregar la columna de acronimo para ahorrar espacio en gráfica
merge_sitios_locs_prio.insert(0, 'acronimo', merge_sitios_locs_prio['estado'].apply(get_acronym))

# eliminar la columna de estado
merge_sitios_locs_prio = merge_sitios_locs_prio.drop(columns=['estado'])

# guardar segmentación 
save_df_in_sheet_from_workbook(merge_sitios_locs_prio, 'sitios_locs_prio', wb_final_results)

ttl_sitios_locs_prio_original = '{:,}'.format(merge_sitios_locs_prio['conteo_prioritarias_original'].sum())
ttl_sitios_locs_prio_merge = '{:,}'.format(merge_sitios_locs_prio['conteo_prioritarias_segmentado'].sum())

# **************************************** CREACIÓN DE GRÁFICO **************************************************************
# crear instancia de gráfico
chrt_merge_sitios_locs_prio = merge_sitios_locs_prio.plot(
	kind='bar', title='Sitios en localidades prioritarias por estado', figsize=(20, 5), width= 0.8, xlabel='Estado de la Rep. Mexicana', ylabel='Sitios por estado', color = ['#691C32', '#BC955C']
)

# cambiar el tamaño de la fuente del las etiquetas de las barras del eje x
chrt_merge_sitios_locs_prio.set_xticklabels(labels=merge_sitios_locs_prio['acronimo'], fontsize=10)

# añadir valores a las columnas 
for x in chrt_merge_sitios_locs_prio.containers:
  chrt_merge_sitios_locs_prio.bar_label(x, fontsize=9)

# personalizar los leyendas 
grupo_nombres = [f'Sitios en locs. prioritarias ORIGINAL ({ttl_sitios_locs_prio_original})', f'Sitios en locs. prioritarias SEGMENTADO ({ttl_sitios_locs_prio_merge})']
chrt_merge_sitios_locs_prio.legend(grupo_nombres, loc='best')

plt.show()

del sitios_locs_no_prio_original, sitios_locs_no_prio_segmentado, merge_sitios_locs_prio, ttl_sitios_locs_prio_original, ttl_sitios_locs_prio_merge, chrt_merge_sitios_locs_prio, grupo_nombres

In [ ]:
# 		╔═╗╦╔╦╗╦╔═╗╔═╗  ╦  ╔═╗╔═╗╔═╗   ╔═╗╦═╗╦╔═╗   ╦ ╦  ╔╦╗╦ ╦ ╦╔═╗╦═╗╔═╗╔═╗    ╔═╗╔═╗╔╗╔  ╔═╗╦    ╔╦╗╔═╗╔╗╔╔═╗╔═╗  ╦  ╔═╗  ╔╦╗╦╔╦╗╔═╗╔╦╗
# 		╚═╗║ ║ ║║ ║╚═╗  ║  ║ ║║  ╚═╗   ╠═╝╠╦╝║║ ║   ╚╦╝  ║║║║ ║ ║║╣ ╠╦╝║╣ ╚═╗    ╚═╗║ ║║║║  ╠═╣║    ║║║║╣ ║║║║ ║╚═╗  ║  ╠═╣  ║║║║ ║ ╠═╣ ║║
# 		╚═╝╩ ╩ ╩╚═╝╚═╝  ╩═╝╚═╝╚═╝╚═╝o  ╩  ╩╚═╩╚═╝o   ╩   ╩ ╩╚═╝╚╝╚═╝╩╚═╚═╝╚═╝    ╚═╝╚═╝╝╚╝  ╩ ╩╩═╝  ╩ ╩╚═╝╝╚╝╚═╝╚═╝  ╩═╝╩ ╩  ╩ ╩╩ ╩ ╩ ╩═╩╝

# segmentación de información donde las localidades sean prioritarias y la pobla
sitios_loc_prio_y_pop_fem_ge50_origin = df[
    (df['porcentaje_pop_fem'] >= 50) & (df['status_prioridad'] == 'Prioritaria')
    ]['Estado*'].value_counts().rename_axis('Estado').reset_index(name='conteo_original')


sitios_loc_prio_y_pop_fem_ge50_segment = archivo_segmentado[
    (archivo_segmentado['porcentaje_pop_fem'] >= 50) & (archivo_segmentado['status_prioridad'] == 'Prioritaria')
    ]['Estado*'].value_counts().rename_axis('Estado').reset_index(name='conteo_segmentado')

# unir los dataframes de conteos de sitios prioritarios con pob. fem >= 50 en df original y segmentado
merge_sitios_loc_prio_y_pop_fem_ge50 = pd.merge(left=sitios_loc_prio_y_pop_fem_ge50_origin, right=sitios_loc_prio_y_pop_fem_ge50_segment, on='Estado')

# agregar la columna de acronimo 
merge_sitios_loc_prio_y_pop_fem_ge50.insert(0, 'acronimo', merge_sitios_loc_prio_y_pop_fem_ge50['Estado'].apply(get_acronym))

# eliminar la columna de estado
merge_sitios_loc_prio_y_pop_fem_ge50 = merge_sitios_loc_prio_y_pop_fem_ge50.drop(columns=['Estado'])

# guardar la segmentación
save_df_in_sheet_from_workbook(merge_sitios_loc_prio_y_pop_fem_ge50, 'sitios_locs_prio_pob_fem_ge50', wb_final_results)

ttl_merge_sitios_loc_prio_y_pop_fem_ge50_original = '{:,}'.format(merge_sitios_loc_prio_y_pop_fem_ge50['conteo_original'].sum())
ttl_merge_sitios_loc_prio_y_pop_fem_ge50_segmentado = '{:,}'.format(merge_sitios_loc_prio_y_pop_fem_ge50['conteo_segmentado'].sum())

# *********** CREACIÓN DEL GRÁFICO ************************

# crear la instancia del gráfico 
chrt_merge_sitios_loc_prio_y_pop_fem_ge50 = merge_sitios_loc_prio_y_pop_fem_ge50.plot(
    kind='bar', title='Sitios en locs. prioritarias con pob. femenina >= 50%', figsize=(20,5), xlabel='Edo. de la República Mexicana', ylabel='No. de sitios prioritarios', color=['#691C32', '#BC955C']
)

# añadir los valores a las columnas
for x in chrt_merge_sitios_loc_prio_y_pop_fem_ge50.containers:
    chrt_merge_sitios_loc_prio_y_pop_fem_ge50.bar_label(x)

# cambiar el tamaño de la fuente del las etiquetas de las barras del eje x
chrt_merge_sitios_loc_prio_y_pop_fem_ge50.set_xticklabels(labels=merge_sitios_loc_prio_y_pop_fem_ge50['acronimo'], fontsize=10)

# cambiar las leyendas 
new_leyends = [
    f'Sitios en locs. prioritarias y pob. fem >= 50% ({ttl_merge_sitios_loc_prio_y_pop_fem_ge50_original} sitios) ORIGINAL',
    f'Sitios en locs. prioritarias y pob. fem >= 50% ({ttl_merge_sitios_loc_prio_y_pop_fem_ge50_segmentado} sitios) SEGMENTADO']
chrt_merge_sitios_loc_prio_y_pop_fem_ge50.legend(new_leyends, loc='best')

plt.show()

del chrt_merge_sitios_loc_prio_y_pop_fem_ge50, sitios_loc_prio_y_pop_fem_ge50_origin, sitios_loc_prio_y_pop_fem_ge50_segment, merge_sitios_loc_prio_y_pop_fem_ge50


In [ ]:
validar_longitud('-99.19822')